In [ ]:
import cv2
import sys
import torch

In [ ]:
from util.load_models import load_netArc, load_face_swap_model, load_insightface_model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from options.test_options import TestOptions

sys.argv = sys.argv[:1]

opt = TestOptions().parse() 

opt.use_mask = True
opt.name = 'people'
opt.Arc_path = './checkpoints/arcface_model/arcface_checkpoint.tar'
opt.crop_size = 224

In [ ]:
netArc = load_netArc(opt.Arc_path, device)
face_swap_model = load_face_swap_model(device, netArc, opt)
detect_model = load_insightface_model(device)

In [ ]:
source_image_path = 'demo_file/source_image/Iron_man.jpg'
target_video_path = 'demo_file/target_video/jirou_anni.mp4'
result_video_path = 'demo_file/result'

## Source图像处理

In [ ]:
source_image = cv2.imread(source_image_path)

In [ ]:
from util.face_process import crop_and_align_face
from util.data_process import process_latent_id

source_crop_align_image, source_M = crop_and_align_face(detect_model, source_image, opt.crop_size)
latend_id = process_latent_id(source_crop_align_image, netArc, device)

## 视频处理

In [ ]:
target_video = cv2.VideoCapture(target_video_path)
target_video.set(cv2.CAP_PROP_POS_FRAMES, 150)

ret, frame = target_video.read()

In [ ]:
target_crop_align_image, target_M = crop_and_align_face(detect_model, frame, opt.crop_size)

In [ ]:
from util.data_process import process_image_tensor

In [ ]:
target_crop_align_tensor = process_image_tensor(target_crop_align_image)

## 人脸交换

In [ ]:
swap_result = face_swap_model(None, target_crop_align_tensor, latend_id, None, True)